Conversion of Earliest Deadline First implementation to SimPy.

In [2]:
! pip install simpy
import simpy

Each task has:
- an id
- an environment
- a resource 
- a waiting time, before beginning the task
- a priority
- and a duration

The task actor/ process starts by waiting a certain amount of time, after which it repeatedly checks if the duration of task execution has been reached. If execution length is not reached, the task requests the resource and tries to timeout for the duration it needs to execute. If it is interrupted during the timeout, the duration of execution is saved and the task will request the resource again.

In [10]:
def task (id, environment, resource, wait, prio, duration, schedule):
    time_spent_executing = 0
    print(f'Task {id} came into existence at {environment.now}')
    yield environment.timeout(wait)

    while(time_spent_executing < duration): #does while loop make sense

        with resource.request(priority = prio) as req:
           
            try:
                print(f'Task {id} requesting at {environment.now} with priority {prio}')
                yield req
                print(f'Task {id} got resource at {environment.now}')
                schedule.append(f'Task {id} at {environment.now}')
                yield environment.timeout(duration-time_spent_executing) 
                break
            except simpy.Interrupt as interrupt:
                print(interrupt)
                print(interrupt.cause)
                by = interrupt.cause.by
                usage = environment.now - interrupt.cause.usage_since
                time_spent_executing += usage
                print(f'Task {id} got preempted by {by} at {environment.now} after {usage} time entities')

    print(f'Task {id} completed at {environment.now}')

In this example the cpu is the preemptive resource all tasks have to share. 

In [11]:
env = simpy.Environment()   
cpu = simpy.PreemptiveResource(env, capacity = 1)
schedule = []

procs  = [env.process(task(1, env, cpu, 0, 0, 5, schedule)),
          env.process(task(2, env, cpu, 0, -1, 5, schedule )),
         env.process(task(3, env, cpu, 0, -2, 5, schedule))]

env.run()

Task 1 came into existence at 0
Task 2 came into existence at 0
Task 3 came into existence at 0
Task 1 requesting at 0 with priority 0
Task 2 requesting at 0 with priority -1
Interrupt(<simpy.resources.resource.Preempted object at 0x10f098d00>)
Task 1 got preempted by <Process(task) object at 0x10f09b940> at 0 after 0 time entities
Task 1 requesting at 0 with priority 0
Task 3 requesting at 0 with priority -2
Interrupt(<simpy.resources.resource.Preempted object at 0x10f098d00>)
Task 2 got preempted by <Process(task) object at 0x10f0985b0> at 0 after 0 time entities
Task 2 requesting at 0 with priority -1
Task 3 got resource at 0
Task 3 completed at 5
Task 2 got resource at 5
Task 2 completed at 10
Task 1 got resource at 10
Task 1 completed at 15


In [ ]:
simpy.interrupt()

Human readable order in which the tasks are executed

In [5]:
def printSchedule(schedule):
    print('The Tasks are executed in the following order:')
    for i in schedule:
        print(i)

printSchedule(schedule)

The Tasks are executed in the following order:
Task 3 at 0
Task 2 at 5
Task 1 at 10
